In [26]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd


In [27]:
model = gp.Model("VRPLTT")

In [28]:
Q_min = 140
Q_max = 300
n_levels = 4
load_levels = np.array([(0.5+i)*(Q_max-Q_min)/n_levels for i in range(n_levels)])
upper = load_levels+(Q_max-Q_min)/(n_levels*2)
lower = load_levels-(Q_max-Q_min)/(n_levels*2)

In [29]:
load_levels

array([ 20.,  60., 100., 140.])

In [30]:
upper

array([ 40.,  80., 120., 160.])

In [37]:
def vel(m, h, P):
    ### aer. resistence
    Cd = 1.18
    A = 0.83 #m^2
    rho = 1.18 #kg/m^3

    c_Fd = rho*Cd*A*0.5

    ### rolling resistance
    Cr = 0.01
    g = 9.81 #m/s^2
    
    eta = 0.95
    
    P_electric = 250 #Watt
    
    c0 = P*eta
    c1 = (m*g* ( Cr*np.cos( np.arctan(h) ) + np.sin( np.arctan(h) ) ))
    c3 = c_Fd
    
    v_max = 30 #km/h
   ### closed form solution of 3-rd degree polynomial

    # p = c1/c3
    # q = -c0/c3
    # print(np.roots([1, 0, p, q]))
    
    ###
    coefs = [c3, 0, c1, -eta*P]
    coefs_electric = [c3, 0, c1, -eta*(P+P_electric)]        
    v = np.max(np.real(3.6*np.roots(coefs)))
    v_electric = np.max(np.real(3.6*np.roots(coefs_electric)))
    if v > 25:
        #print('up-cyclist only')
        return min(v, v_max)
    else:
        #print('up-electric')
        return min(v_electric, 25)


def time_matrix(data_matrix, load_levels,P):
    N = len(data_matrix.index)
    elevation = data_matrix.elevation.to_numpy()
    d_ij_mat = data_matrix.iloc[:, 8:].to_numpy()
    t_ij_mat = np.zeros((N, N, n_levels))
    for i in range(N):
        for j in range(N):
            if i!=j:
                d_ij = d_ij_mat[i, j]
                h = (elevation[j] - elevation[i])/d_ij
                for l in range(n_levels):
                    m = load_levels[l]
                    print(vel(m,h,P))
                    t_ij_mat[i, j, l] =60*d_ij/vel(m, h, P)
    return np.round(t_ij_mat,4)

The forces acting on the bicyle are: the aerodynamic drag resistance $F_{D}$, the rolling resistance $F_{R}$ and the component of the gravity force along the direction of motion $F_{G}$:

* $ F_{D} = \frac{\rho C_{D}Av^{2}}{2}$
* $ F_{R} = C_{R}mg cos(arctan(h))$
* $ F_{G} = mg sin(arctan(h))$

We have to distinguish two cases: uphill and downhill.
In uphill, the component of the gravity force along the direction of motion is a resistance force, while in downhill it helps the motion of the bicyle.
so we have, at equilibrium:

* $\mathrm{ \eta P_{traction} = (F_{D} + (F_{R} + F_{G})v }$

where $\eta$ is the efficiency of the mechanic transmission, which is assumed to be 0.95.

If we use a negative h in the formuals above when traversing an edge with negative slope, since $cos(\alpha) = cos(-\alpha)$ and $sin(\alpha) = -sin(-\alpha)$, we have just one expression for the equation to solve.

Moreover, we assume that the bicyle speed is limited in downhill for safety reasons. Within cities, a typical speed limit for for motor vehicles is 50km/h. Considering also that the regulation for pedelec bicyle allows a maximum speed of 25km/h in electric assisted mode, we set a maximum speed in downhill of 30km/h.
The delta energy can be used for recharging the battery for example, similarly to other solutions in the automaotive industry.
The maximum speed limit is imposed on uphill as well.
The strategy for solving the equation is:
* solve the equation for the two cases: (1) $P=P_{cyclist}$, and (2) $P=P_{cyclist}+P_{electric}$ 
* If the cyclist can reach a speed that is higher than the speed allowed in electric mode, use that speed, subject to the limit of maximum speed allowed,
* otherwise use the minimum value among 25 km/h and the solution of problem (2)

In practice we have $P_{electric}$ = 250W, while for $\overline {P_{cyclist}}$ we assume to have few different types of cyclist.

In [38]:
print('base model v=', vel(150, 0.05, 150+250))
print('new model v=', vel1(150, 0.05, 150))
print('\n')
print('base model v=', vel(150, -0.05, 150+250))
print('new model v=', vel1(150, -0.05, 150))
print('\n')
print('base model v=', vel(150, -0.01, 150+250))
print('new model v=', vel1(150, -0.01, 150))
print('\n')
print('base model v=', vel(150, -0.01, 300+250))
print('new model v=', vel1(150, -0.01, 300))
print('\n')
print('base model v=', vel(150, 0.01, 450+250))
print('new model v=', vel1(150, 0.01, 450))

base model v= 20.71513732148457
new model v= None


base model v= 30
new model v= None


base model v= 30
new model v= None


base model v= 30
new model v= None


base model v= 30
new model v= None


In [39]:
df = pd.read_csv("./instances/small/Fukuoka_01.csv")
P = 500 # Watt

t=time_matrix(df,load_levels,P=P)

30
30
30
30
17.8394574032708
6.565137524091341
3.9542988327754443
2.826428414584897
13.128455986420075
4.547879758717003
2.7322278902668
1.9520311017177538
12.825578760322347
4.431234968489084
2.6618953824040683
1.9017500356542438
13.02805355867234
4.509083099578753
2.708831933985499
1.935304834800708
13.311044578668609
4.618772873359898
2.774987253047358
1.9826016844038656
12.840801479665767
4.4370700105957415
2.665413062006365
1.904264762305013
12.646477004197896
4.362795988227109
2.620641516084615
1.8722589933221327
12.626266422282967
4.355097355836362
2.616001463415903
1.868942041027002
12.618393052905143
4.352099542388857
2.6141946773149396
1.8676504596143453
12.62273477515069
4.353752577393484
2.615190961611855
1.8683626533856783
12.619070468564193
4.352357441824289
2.6143501129851794
1.8677615727518186
12.622786370886024
4.353772222917752
2.61520280200112
1.8683711174911155
12.631286288469715
4.3570090729838435
2.617153666598415
1.869765694811369
12.618581082630051
4.35217112678

In [40]:
from pprint import pprint
print(t[3,4])
print(t[4,3])


[0.7867 2.2803 3.7961 5.3135]
[0.332 0.332 0.332 0.332]


In [35]:
def extract_min(Q,distance):
    minimo=float("inf") #highest python value
    nome=""
    for q in Q: #find the smallest
        if distance[q] <= minimo:
            nome=q
            minimo=distance[q]
    Q.remove(nome) #remove the correct node from the que
    return nome # tell me wich node is removed

def W(u,v,weight_matrix):
    for i in adj[u]:
        if i.end==v:
            return i.weight
        
def dijkstra(source,weight_matrix,l=0):
    distance=dict() #dict for distance from source
    parent=dict() #dict for parentnes of each node
    nodes=np.array([i for i in range(len(weight_matrix))]) #list of current node to iterate
    for node in nodes:
        distance[node]=float("inf") 
        parent[node]=None
    distance[source]=0
    S=[]
    Q={node for node in nodes} #set used ad que, coupled with distance dict
    while len(Q)>0:
        u=extract_min(Q,distance) #extract the min from Q,using distance dict
        S.append(u) #ultimated nodes
        for v,w in enumerate(weight_matrix[u]): #for each node v, outgoing from u, with weight w
            if distance[v] > distance[u] + w[l]: #relax phase
                distance[v]= distance[u] + w[l]
                parent[v]=u
        #print(f"{u = }")
        #print(f"{distance = }")
    return distance,parent #return distances from source and each node parent !
            
dist,parent = dijkstra(0,t)
pprint(dist)
pprint(parent)

{0: 0,
 1: 0.38,
 2: 1.1314,
 3: 1.874,
 4: 2.6607000000000003,
 5: 2.9607,
 6: 3.3047,
 7: 4.2781,
 8: 4.2905,
 9: 5.0012,
 10: 4.3412,
 11: 4.0361,
 12: 2.3345,
 13: 2.2343,
 14: 3.6325,
 15: 4.2317,
 16: 4.6217,
 17: 1.7052,
 18: 1.333,
 19: 0.7675,
 20: 1.472}
{0: None,
 1: 0,
 2: 1,
 3: 2,
 4: 3,
 5: 4,
 6: 5,
 7: 6,
 8: 12,
 9: 10,
 10: 1,
 11: 1,
 12: 1,
 13: 1,
 14: 12,
 15: 1,
 16: 14,
 17: 0,
 18: 0,
 19: 0,
 20: 19}


In [36]:
for i in t[:,:,0]:
    for j in i:
        print(f"\t{round(j,3)}",end=" ")
    print("\n")

	0.0 	0.38 	1.17 	2.308 	3.139 	3.781 	4.48 	5.602 	6.248 	6.434 	4.864 	4.558 	2.862 	2.462 	4.099 	4.56 	5.039 	1.705 	1.333 	0.768 	1.52 

	0.38 	0.0 	0.751 	1.575 	2.371 	3.043 	3.78 	4.836 	5.351 	5.533 	3.961 	3.656 	1.954 	1.854 	3.711 	3.852 	4.792 	2.143 	1.948 	1.08 	1.34 

	0.696 	0.36 	0.0 	0.743 	1.53 	2.22 	2.984 	4.008 	4.603 	5.129 	3.719 	3.219 	1.717 	1.612 	3.469 	3.609 	4.55 	1.962 	1.848 	0.824 	1.557 

	1.01 	0.674 	0.314 	0.0 	0.787 	1.468 	2.154 	3.264 	3.915 	4.44 	3.78 	2.53 	1.855 	2.148 	3.785 	3.852 	4.778 	1.488 	2.504 	1.138 	0.988 

	1.342 	1.006 	0.646 	0.332 	0.0 	0.3 	0.646 	2.463 	3.383 	4.449 	3.79 	2.539 	2.643 	2.934 	4.567 	4.64 	5.565 	1.82 	1.43 	1.47 	1.32 

	1.642 	1.306 	0.946 	0.632 	0.675 	0.0 	0.344 	1.78 	2.671 	3.842 	3.229 	2.934 	3.356 	3.643 	5.274 	4.422 	5.451 	2.122 	2.204 	1.772 	1.62 

	1.988 	1.65 	1.292 	0.978 	1.379 	0.649 	0.0 	0.973 	1.85 	3.124 	3.11 	3.209 	4.172 	4.453 	5.444 	4.303 	5.333 	2.466 	2.918 	2.116 	1.964 

	

In [11]:
%%timeit
dist,parent = dijkstra(0,t)


291 µs ± 12.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [12]:
def shortest_path_matrix(t_ij_mat):
    sp_matrix=np.empty_like(t_ij_mat)
    nodes=np.array([i for i in range(t_ij_mat.shape[0])])
    levels=np.array([i for i in range(t_ij_mat.shape[-1])])
    print(levels)
    for source in nodes:
        for l in levels:
            distance,_t=dijkstra(source,t_ij_mat,l)
            sp_matrix[source,:,l]=list(distance.values())
    return sp_matrix



In [13]:

sp = shortest_path_matrix(t)
for i in sp[:,:,0]:
    for j in i:
        print(f"\t{round(j,2)}",end=" ")
    print("\n")
    

[0 1 2 3]
	0.0 	0.38 	1.48 	2.57 	3.39 	3.69 	4.04 	5.32 	5.21 	6.87 	6.21 	5.76 	3.26 	3.11 	4.55 	6.05 	6.01 	2.39 	1.96 	1.12 	2.15 

	0.38 	0.0 	1.1 	2.19 	3.35 	3.65 	3.99 	4.94 	4.83 	6.49 	5.83 	5.38 	2.88 	2.73 	4.17 	5.67 	5.63 	2.77 	2.34 	1.5 	1.97 

	0.7 	0.36 	0.0 	1.09 	2.25 	2.55 	2.89 	4.33 	4.48 	6.13 	5.47 	4.74 	2.53 	2.37 	3.82 	5.31 	5.28 	2.58 	2.66 	0.82 	1.86 

	1.01 	0.67 	0.31 	0.0 	1.16 	1.46 	1.8 	3.23 	4.52 	6.22 	5.56 	3.72 	2.73 	2.69 	4.03 	5.63 	5.48 	1.49 	2.59 	1.14 	0.99 

	1.34 	1.01 	0.65 	0.33 	0.0 	0.3 	0.64 	2.08 	3.37 	5.24 	5.05 	3.74 	3.06 	3.02 	4.36 	5.76 	5.82 	1.82 	1.43 	1.47 	1.32 

	1.64 	1.31 	0.95 	0.63 	0.99 	0.0 	0.34 	1.78 	3.07 	4.94 	4.75 	4.32 	3.36 	3.32 	4.66 	5.46 	5.89 	2.12 	2.42 	1.77 	1.62 

	1.99 	1.65 	1.29 	0.98 	1.93 	0.94 	0.0 	1.43 	2.72 	4.6 	4.58 	4.7 	3.71 	3.66 	4.59 	5.28 	5.72 	2.47 	3.36 	2.12 	1.96 

	2.4 	2.06 	1.7 	1.39 	1.06 	0.75 	0.41 	0.0 	1.79 	3.67 	4.03 	4.56 	4.12 	3.7 	3.66 	4.74 	5.12 	2.87 	2.4